In [3]:
# Add ./python/ to path
import sys
sys.path.append('../data/')


from data_prep import *
from charts import *
from players import *
from video_analysis import *
from team_sheets import *

# # Load data
# game_df = team_sheets()
# players_df = players(game_df)
# players_agg_df = players_agg(players_df)
# lineouts_df = lineouts()
# set_piece_df = set_piece_results()
# analysis = game_stats()
# players_table = players_table_data(players_df, players_agg_df)
 
# # Save data
# game_df.to_csv('data/game.csv', index=False)
# players_df.to_csv('data/players.csv', index=False)
# players_agg_df.to_csv('data/players_agg.csv', index=False)
# lineouts_df.to_csv('data/lineouts.csv', index=False)
# set_piece_df.to_csv('data/set_piece.csv', index=False)
# analysis.to_csv('data/analysis.csv', index=False)
# update_season_summaries(game_df, seasons=["2024/25"])


game_df = pd.read_json('../data/game.json')
players_df = pd.read_json('../data/players.json')
players_agg_df = pd.read_json('../data/players_agg.json')
lineouts_df = pd.read_json('../data/lineouts.json')
set_piece_df = pd.read_json('../data/set_piece.json')
analysis = pd.read_json('../data/analysis.json')
players_table = pd.read_json('../data/player_table.json')

# # One-off charts (only source data needs updating)
# captains_chart(file='Charts/captains.html')
# results_chart(file='Charts/results.html')
# plot_games_by_player(file='Charts/appearances.html')
# plot_starts_by_position(file='Charts/positions.html')
card_chart(file='../Charts/cards.html')
# points_scorers_chart(file='Charts/points.html')
# team_sheets_chart(file='Charts/team-sheets.html')
# set_piece_h2h_chart(file='Charts/set-piece.html')
# squad_continuity_chart(file='Charts/continuity.html')

# # Self-contained charts (chart needs updating)
# game_stats_charts(analysis, file='Charts/video_analysis.html')
# lineout_success(types=types, file='Charts/lineouts.html')

Updated ../Charts/cards.html


alt.Chart(...)

# Individual Player Stats

- Table:
  - Position
  - Debut
  - Total Games (this season/total)
  - Tries (this season/total)
- Charts:
  - Games per season (color by squad)
  - Games by position (color by squad)
  - Tries/points per season

In [2]:
player_profile_charts(df=players_df, df_agg=players_agg_df)

Updated Charts/player_appearances.html


alt.VConcatChart(...)

# Team Sheets

In [ ]:
def team_sheet_chart(
        squad=1, 
        names=None, 
        captain=None, 
        vc=None, 
        opposition=None, 
        home=True, 
        competition="Counties 1 Sussex",
        season="2023/24"
    ):

    if names is None:
        df = team_sheets()    

        # Last row as dict
        team = df.iloc[-1].to_dict()


        label = f'{"1st" if squad==1 else "2nd"} XV vs {team["Opposition"]}({team["Home/Away"]})'
        captain = team["Captain"]
        vc = team["VC1"]
        season = team["Season"]
        competition = team["Competition"]

        # Keep keys that can be converted to integers
        team = {int(k): v for k, v in team.items() if k.isnumeric() and v}

        # Convert team to dataframe with Number and Player columns
        team = pd.DataFrame(team.items(), columns=["Number", "Player"])

    else:
        label = f'{"1st" if squad==1 else "2nd"} XV vs {opposition} ({"H" if home else "A"})'

        # Convert names to Player column of a dataframe with Number column (1-len(names))
        team = pd.DataFrame({"Player": names, "Number": range(1, len(names)+1)})

    coords = pd.DataFrame([
                {"n": 1, "x": 10, "y": 81},
                {"n": 2, "x": 25, "y": 81},
                {"n": 3, "x": 40, "y": 81},
                {"n": 4, "x": 18, "y": 69},
                {"n": 5, "x": 32, "y": 69},
                {"n": 6, "x": 6, "y": 61},
                {"n": 7, "x": 44, "y": 61},
                {"n": 8, "x": 25, "y": 56},
                {"n": 9, "x": 20, "y": 42},
                {"n": 10, "x": 38, "y": 36},
                {"n": 11, "x": 8, "y": 18},
                {"n": 12, "x": 56, "y": 30},
                {"n": 13, "x": 74, "y": 24},
                {"n": 14, "x": 92, "y": 18},
                {"n": 15, "x": 50, "y": 10},
                {"n": 16, "x": 80, "y": 82},
                {"n": 17, "x": 80, "y": 74},
                {"n": 18, "x": 80, "y": 66},
                {"n": 19, "x": 80, "y": 58},
                {"n": 20, "x": 80, "y": 50},
                {"n": 21, "x": 80, "y": 42},
                {"n": 22, "x": 80, "y": 34},
                {"n": 23, "x": 80, "y": 26},
            ])
    team = team.merge(coords, left_on="Number", right_on="n", how="inner").drop(columns="n")

    # Add captain (C) and vice captain (VC) else None
    team["Captain"] = team["Player"].apply(lambda x: "C" if x == captain else "VC" if x == vc else None)

    team["Player"] = team["Player"].str.split(" ")

    team.to_dict(orient="records")

    with open("team-sheet-lineup.json") as f:
        chart = json.load(f)
    chart["data"]["values"] = team.to_dict(orient="records")
    chart["title"]["text"] = label
    chart["title"]["subtitle"] = f"{season} - {competition}"

    n_replacements = len(team) - 15
    
    y = 126 + (n_replacements * 64)
    chart["layer"][0]["mark"]["y2"] = y
    # return chart
    return alt.Chart.from_dict(chart)

team_sheet_chart()

alt.LayerChart(...)

In [175]:
import requests

url = 'https://www.englandrugby.com/fixtures-and-results/search-results?competition=261&season=2024-2025&division=56612#results'
response = requests.get(url)

# find all <a> tags with a link containing '/fixtures-and-results/match-centre-community'
soup = BeautifulSoup(response.text, 'html.parser')
links = [a['href'] for a in soup.find_all('a', href=True) if '/fixtures-and-results/match-centre-community' in a['href']]

ids = [link.split("=")[-1] for link in links]

with open('../data/match_ids.json', 'w') as f:
    json.dump(ids, f)

In [16]:
import requests
from bs4 import BeautifulSoup

def get_players(soup):

    numbers = soup.find_all(class_='c085-lineup-table-player-number')
    numbers = [number.text.strip() for number in numbers]

    players = soup.find_all(class_='c085-lineup-table-player-name-text')
    players = [player.text.strip() for player in players]
    players = list(dict.fromkeys(players))
    
    split_index = len(numbers) - numbers[::-1].index('15') - 1
    
    home_players = {n:" ".join(p.split()) for n,p in zip(numbers[:split_index], players[:split_index])}
    away_players = {n:" ".join(p.split()) for n,p in zip(numbers[split_index:], players[split_index:])}

    return [home_players, away_players]

def get_match_info(id):

    url = f'https://www.englandrugby.com/fixtures-and-results/match-centre-community?matchId={id}#lineup'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    score = soup.find(class_='c042-match-score')
    if score is None:
        return None
    
    score = [int(s) for s in score.text.strip().split(' - ')]

    date = soup.find(id='c042-event-date').text.strip()
    date = date[:date.rfind('202') + 4]
    date = pd.to_datetime(date, format="%A %d %B %Y").strftime("%Y-%m-%d")

    teams = [team.text.strip() for team in soup.find_all(class_='c042-team-name')]

    logos = [img['src'] for img in soup.find_all('img', class_='c042-team-logo')]

    players = get_players(soup)

    match_data = {
        "date": date,
        "score": score,
        "teams": teams,
        "logos": logos,
        "players": players
    }   

    # Save to json
    with open(f'../data/match_data/{id}.json', 'w') as f:
        json.dump(match_data, f)

    return {
        "date": date,
        "score": score,
        "teams": teams,
        "logos": logos,
        "players": players
    }

In [17]:
# for id in ids[50:]:
#     if not os.path.exists(f'data/match_data/{id}.json'):
#         get_match_info(id)

get_match_info(1136512)

{'date': '2025-03-01',
 'score': [24, 17],
 'teams': ['Eastbourne', 'Cobham'],
 'logos': ['https://images.englandrugby.com/club_images/7209.png',
  'https://images.englandrugby.com/club_images/5336.png'],
 'players': [{'15': 'Mason Dowle',
   '14': 'Louis Wood',
   '13': 'Jake Howe',
   '12': 'Maxence Saint-André',
   '11': 'Stuart Baker',
   '10': 'Aaron Hossack',
   '9': 'Leon Wood',
   '8': 'Jeremy Montes',
   '7': 'Owen Davies',
   '6': 'Jack Curtis',
   '5': 'Luke Thomas',
   '4': 'Joshua Buckwell',
   '3': 'Paul Smith',
   '2': 'Konrad Oneill',
   '1': 'Christopher Hosking',
   'S1': 'Joseph Bettles',
   'S2': 'Carl Slaughter',
   'S3': 'Brandon Foot'},
  {'15': 'Sandro Celebidachi',
   '14': 'Michael Osner',
   '13': 'Henry MacQuire',
   '12': 'Will Palmer',
   '11': 'Ross Munro',
   '10': 'Will Stephen',
   '9': 'Rory Penfold',
   '8': 'Jake Mcquade',
   '7': 'Oliver Fielding',
   '6': 'Thomas Davison',
   '5': 'Louis Porter',
   '4': 'Luke Westgarth-Taylor',
   '3': 'Ryan Aujl

In [51]:
import json
import os
import pandas as pd

# Define file paths
MATCH_DATA_FOLDER = "../data/match_data"  # Adjust if needed
OUTPUT_FILE = "season_squad_analysis.csv"

# List to store match data
data = []

# Process each match file
# For each file in data/match_data/ folder
for match_file in os.listdir(MATCH_DATA_FOLDER):

    with open(os.path.join(MATCH_DATA_FOLDER, match_file)) as f:
        # Load match data
        # print(match_file)
        match_data = json.load(f)

    # Extract match details
    match_date = match_data["date"]
    teams = match_data["teams"]
    players = match_data["players"]

    for i in range(2):  # Loop over both teams
        team_name = teams[i]
        
        for position, player_name in players[i].items():
            data.append({
                "Match ID": int(match_file[:-5]),
                "Date": match_date,
                "Team": team_name,
                "Player": player_name,
                "Position": position
            })

# Create DataFrame
df = pd.DataFrame(data)
df["Unit"] = df["Position"].apply(lambda x: "Bench" if not(x.isnumeric()) else "Forwards" if int(x) <= 8 else "Backs")

df

players_per_team = (
    df.groupby(["Team", "Player"])["Unit"]
    .agg(lambda x: "Forwards" if "Forwards" in x.values else "Backs" if "Backs" in x.values else "Bench")
    .reset_index()
    .groupby(["Team", "Unit"])
    .size()
    .reset_index()
)

total_players_per_team = df.groupby("Team")["Player"].nunique().reset_index()
total_players_per_team.columns = ["Team", "Total Players"]
total_players_per_team["Unit"] = "Total"

players_per_team = pd.concat([players_per_team, total_players_per_team], ignore_index=True)

retention_data = []

# Process retention per team
for team, matches in df.groupby("Team"):
    prev_squad = set()
    prev_forwards = set()
    prev_backs = set()
    
    for match_id, match in matches.groupby("Match ID"):
        current_squad = set(match[match["Unit"]!="Bench"]["Player"])  # Exclude substitutes
        forwards = set(match[match["Unit"]=="Forwards"]["Player"])
        backs = set(match[match["Unit"]=="Backs"]["Player"])
        
        if prev_squad:
            retained = len(current_squad & prev_squad)
            retained_forwards = len(forwards & prev_forwards)
            retained_backs = len(backs & prev_backs)
        else:
            retained = None  # No previous match to compare
            retained_forwards = None
            retained_backs = None

        retention_data.append({
            "Match ID": match_id,
            "Date": match["Date"].iloc[0],
            "Team": team,
            "Players Retained": retained,
            "Forwards Retained": retained_forwards,
            "Backs Retained": retained_backs
        })
        
        prev_squad = current_squad
        prev_forwards = forwards
        prev_backs = backs

retention_df = pd.DataFrame(retention_data).dropna()  # Remove first match (no previous squad)

# Average squad retention per team
average_retention = (
    retention_df.groupby("Team").agg(
        {"Players Retained": "mean", "Forwards Retained": "mean", "Backs Retained": "mean"}
    )
    .reset_index()
)

average_retention

,Team,Players Retained,Forwards Retained,Backs Retained
0,Cobham,8.947368,5.263158,3.684211
1,East Grinstead,10.105263,5.473684,4.578947
2,Eastbourne,11.894737,6.210526,5.368421
3,Haywards Heath,8.736842,4.000000,4.157895
4,Hove,9.789474,5.052632,4.473684
5,KCS Old Boys,8.722222,4.777778,3.500000
6,London Cornish,8.444444,4.388889,4.000000
7,Old Haileyburians,6.473684,3.368421,2.578947
8,Old Rutlishians,11.000000,5.789474,5.210526
9,Trinity,10.263158,5.315789,4.631579


In [83]:
colors = {
    "East Grinstead": ["darkblue", "white"],
    "Hove": ["dodgerblue", "maroon"],
    "Eastbourne": ["royalblue", "yellow"],
    "Haywards Heath": ["red", "black"],
    "Old Haileyburians": ["#89273a", "white"],
    "Old Rutlishians": ["gold", "navy"],
    "Trinity": ["lightsteelblue", "darkblue"],
    "Weybridge Vandals": ["purple", "green"],
    "London Cornish": ["black", "gold"],
    "Cobham": ["navy", "crimson"],
    "KCS Old Boys": ["red", "yellow"],
    "Twickenham": ["black", "red"],
}
teams = []
main_colors = []
accent_colors = []

for k, v in colors.items():
    teams.append(k)
    main_colors.append(v[0])
    accent_colors.append(v[1])

selection = alt.selection_point(fields=["Team"], bind="legend", value="East Grinstead")


players_per_team = (
    df.groupby(["Team", "Player"])["Unit"]
    .agg(lambda x: "Forwards" if "Forwards" in x.values else "Backs" if "Backs" in x.values else "Bench")
    .reset_index()
    .groupby(["Team", "Unit"])
    .size()
    .reset_index()
    .rename(columns={0: "Total Players"})
)

total_players_per_team = df.groupby("Team")["Player"].nunique().reset_index()
total_players_per_team.columns = ["Team", "Total Players"]
total_players_per_team["Unit"] = "Total"

players_per_team = pd.concat([players_per_team, total_players_per_team], ignore_index=True)

retention_data = []

# Process retention per team
for team, matches in df.groupby("Team"):
    prev_squad = set()
    prev_forwards = set()
    prev_backs = set()
    
    for match_id, match in matches.groupby("Match ID"):
        current_squad = set(match[match["Unit"]!="Bench"]["Player"])  # Exclude substitutes
        forwards = set(match[match["Unit"]=="Forwards"]["Player"])
        backs = set(match[match["Unit"]=="Backs"]["Player"])
        
        if prev_squad:
            retained = len(current_squad & prev_squad)
            retained_forwards = len(forwards & prev_forwards)
            retained_backs = len(backs & prev_backs)
        else:
            retained = None  # No previous match to compare
            retained_forwards = None
            retained_backs = None

        retention_data.append({
            "Match ID": match_id,
            "Date": match["Date"].iloc[0],
            "Team": team,
            "Players Retained": retained,
            "Forwards Retained": retained_forwards,
            "Backs Retained": retained_backs
        })
        
        prev_squad = current_squad
        prev_forwards = forwards
        prev_backs = backs

retention_df = pd.DataFrame(retention_data).dropna()  # Remove first match (no previous squad)

# Average squad retention per team
average_retention = (
    retention_df
    .groupby("Team").agg({"Players Retained": "mean", "Forwards Retained": "mean", "Backs Retained": "mean"})
    .reset_index()
    .melt("Team", var_name="Unit", value_name="Average Retention")
)

average_retention["Unit"] = average_retention["Unit"].str.replace(" Retained", "").replace("Players", "Total")
average_retention

,Team,Unit,Average Retention
0,Cobham,Total,8.947368
1,East Grinstead,Total,10.105263
2,Eastbourne,Total,11.894737
3,Haywards Heath,Total,8.736842
4,Hove,Total,9.789474
5,KCS Old Boys,Total,8.722222
6,London Cornish,Total,8.444444
7,Old Haileyburians,Total,6.473684
8,Old Rutlishians,Total,11.000000
9,Trinity,Total,10.263158


In [87]:
average_retention_chart = (
    alt.Chart(average_retention)
    .mark_bar()
    .encode(
        x=alt.X("Average Retention:Q", title=None),
        y=alt.Y("Team:N", sort="-x", title=None, axis=alt.Axis(ticks=False, domain=False, labelPadding=10)),
        color=alt.Color("Team:N", scale=alt.Scale(domain=teams, range=main_colors)),
        stroke=alt.Stroke("Team:N", scale=alt.Scale(domain=teams, range=accent_colors), legend=None),
        tooltip=["Team", "Unit:N", "Players Retained:Q"],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
        column=alt.Column("Unit:N", title=None, sort=["Total", "Forwards", "Backs"])
    )
    .resolve_scale(y="independent", x="independent")
    .properties(
        title=alt.Title(
            text="Average Squad Retention",
            subtitle="Average number of players retained from the starting XV from game to game"
        ), 
        width=200, height=alt.Step(30)
    )
    .add_params(selection)
)
average_retention_chart

alt.Chart(...)

In [84]:
# Save to CSV for easy analysis
player_df.to_csv(OUTPUT_FILE, index=False)
print(f"Season squad data saved to {OUTPUT_FILE}")

# Summary Statistics
appearance_count = player_df.groupby(["Team", "Player"]).size().reset_index(name="Appearances").sort_values("Appearances", ascending=False)
players_per_team = player_df.groupby("Team")["Player"].nunique().reset_index(name="Total Players").sort_values("Total Players", ascending=False)

print("\nTop 10 Players by Appearances:")
print(appearance_count)

print("\nTotal Players per Team:")
print(players_per_team)


Season squad data saved to season_squad_analysis.csv

Top 10 Players by Appearances:
                  Team               Player  Appearances
121         Eastbourne            Leon Wood           19
110         Eastbourne  Christopher Hosking           19
521         Twickenham        Jack Fullicks           19
131         Eastbourne           Paul Smith           19
448    Old Rutlishians         LOUIE DAVIES           19
..                 ...                  ...          ...
356  Old Haileyburians        Charles Wells            1
357  Old Haileyburians      Charlie Golding            1
358  Old Haileyburians     Christian Murphy            1
360  Old Haileyburians     Daniel Blackwell            1
582  Weybridge Vandals          Wayne Mason            1

[583 rows x 3 columns]

Total Players per Team:
                 Team  Total Players
7   Old Haileyburians             73
5        KCS Old Boys             61
3      Haywards Heath             54
0              Cobham             

In [164]:
from charts import *
import altair as alt

colors = {
    "East Grinstead": ["darkblue", "white"],
    "Hove": ["dodgerblue", "maroon"],
    "Eastbourne": ["royalblue", "yellow"],
    "Haywards Heath": ["red", "black"],
    "Old Haileyburians": ["#89273a", "white"],
    "Old Rutlishians": ["gold", "navy"],
    "Trinity": ["lightsteelblue", "darkblue"],
    "Weybridge Vandals": ["purple", "green"],
    "London Cornish": ["black", "gold"],
    "Cobham": ["navy", "crimson"],
    "KCS Old Boys": ["red", "yellow"],
    "Twickenham": ["black", "red"],
}
teams = []
main_colors = []
accent_colors = []

for k, v in colors.items():
    teams.append(k)
    main_colors.append(v[0])
    accent_colors.append(v[1])

selection = alt.selection_point(fields=["Team"], bind="legend")

# Altair Chart: Top 10 Players by Appearances
top_players_chart = (
    alt.Chart(appearance_count.sort_values("Appearances", ascending=False).head(12))
    .mark_bar()
    .encode(
        x=alt.X("Appearances:Q", title="Total Appearances"),
        y=alt.Y("Player:N", sort="-x", title="Player"),
        color=alt.Color("Team:N", scale=alt.Scale(domain=teams, range=main_colors), legend=alt.Legend(title="Team")),
        stroke=alt.Stroke("Team:N", scale=alt.Scale(domain=teams, range=accent_colors), legend=None),
        tooltip=["Player", "Team", "Appearances"],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    )
    .properties(title="Most Appearances", width=300, height=alt.Step(30))
    .add_selection(selection)
)

# Altair Chart: Total Players per Team (sorted)
players_per_team_chart = (
    alt.Chart(players_per_team)
    .mark_bar(strokeWidth=2)
    .encode(
        x=alt.X("Total Players:Q", title="Total Players"),
        y=alt.Y("Team:N", sort="-x", title="Team"),
        color=alt.Color("Team:N", scale=alt.Scale(domain=teams, range=main_colors)),
        stroke=alt.Stroke("Team:N", scale=alt.Scale(domain=teams, range=accent_colors), legend=None),
        tooltip=["Team", "Total Players"],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    )
    .properties(title="Squad Size", width=300,height=alt.Step(30))
    .add_selection(selection)
)

# Average squad retention per team
average_retention = retention_df.groupby("Team")["Players Retained"].mean().reset_index(name="Average Retention")

# Altair Chart: Average Squad Retention per Team
average_retention_chart = (
    alt.Chart(average_retention)
    .mark_bar()
    .encode(
        x=alt.X("Average Retention:Q", title="Average Players Retained"),
        y=alt.Y("Team:N", sort="-x", title="Team"),
        color=alt.Color("Team:N", scale=alt.Scale(domain=teams, range=main_colors)),
        stroke=alt.Stroke("Team:N", scale=alt.Scale(domain=teams, range=accent_colors), legend=None),
        tooltip=["Team", "Average Retention"],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    )
    .properties(title="Average Squad Retention", width=300, height=alt.Step(30))
    .add_selection(selection)
)

# Create violin plot data
appearance_count = df.groupby(["Team", "Player"]).size().reset_index(name="Appearances")

# Violin Plot for Player Appearance Distribution
violin_chart = (
    alt.Chart(appearance_count)
    .transform_density(
        density="Appearances",
        groupby=["Team"],
        as_=["Appearances", "Density"]
    ) 
    .mark_area(orient="horizontal", opacity=0.5)
    .encode(
        y=alt.Y("Appearances:Q", title="Player Appearances"),
        x=alt.X("Density:Q", title="Density", stack="center", axis=alt.Axis(ticks=False, labels=False, offset=10)),
        color=alt.Color("Team:N", scale=alt.Scale(domain=teams, range=main_colors)),
        stroke=alt.Stroke("Team:N", scale=alt.Scale(domain=teams, range=accent_colors), legend=None),
        tooltip=["Team", "Appearances"],
        facet=alt.Facet("Team:N", columns=4, title=None, header=alt.Header(labelColor="gray"), spacing={"row": 30}),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    )
    .properties(
        title=alt.Title(
            text="Player Appearance Distribution", 
            subtitle=["Distribution of appearances per player for each team, from most appearances (top) to least (bottom).", "The wider the area, the more players with that number of appearances. "],
        ),
        width=300, height=200)
    .add_selection(selection)
)

# Display charts
chart = (
    ((top_players_chart | players_per_team_chart | average_retention_chart) & violin_chart)
    .configure_scale(bandPaddingInner=0.2).resolve_scale(color="shared")
)

chart.save("charts/league_squad_analysis.html")

/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_44170/3866265683.py:42: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  .add_selection(selection)
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_44170/3866265683.py:58: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  .add_selection(selection)
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_44170/3866265683.py:77: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  .add_selection(selection)
/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_44170/3866265683.py:107: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  .add_selection(selection)


FileNotFoundError: [Errno 2] No such file or directory: 'charts/league_squad_analysis.html'

In [ ]:
retention_chart = (
    alt.Chart(retention_df)
    .mark_line(point={"size": 50})
    .encode(
        x=alt.X("Date:T", title="Match Date"),
        y=alt.Y("Players Retained:Q", title="Players Retained", scale=alt.Scale(domain=[0, 15])),
        color=alt.Color("Team:N", scale=alt.Scale(domain=teams, range=main_colors)),
        tooltip=["Team", "Date", "Players Retained"],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .properties(
        title=alt.Title(
            text="Squad Retention Over Time",
            subtitle="Number of players retained in the starting XV for the following match."
        ),
        width=800,
        height=400
    )
    .add_params(selection)
)

# Display the chart
retention_chart

alt.Chart(...)